## Bayesian Logistic Regression for IMDB 50K Dataset

### Data Pre-Processing

In [5]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

imdb = pd.read_csv("IMDB Dataset.csv")
imdb.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [7]:
imdb.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


#### Remove Stop Words and other Rare Words

In [4]:
#list of stopwords from NLTK
from cgitb import text

# stopword_list=nltk.corpus.stopwords.words('english')

#remove HTML tags
def strip_html(html_text: str):
    return BeautifulSoup(html_text, 'lxml').get_text()

#make sure to add more cleaning (remove punctuation, )
def clean_review(review: str):
    return strip_html(review)

#clean each review in dataset
imdb.review.apply(clean_review)

#tags should be removed from the 1st review
imdb.head(2)

/Users/seichang/miniconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive


#### Bag of Words Vectorized Representation

In [ ]:
#Use CountVectorizer to extract BOW representation

cv = CountVectorizer()

#### Train, Test, Validation Split


In [22]:
#80% training, 20% testing split
X_train, X_test, y_train, y_test = train_test_split(imdb.review, imdb.sentiment, test_size=0.2, random_state=32)
print("{} {}\n{} {}".format(
    X_train.shape,
    y_train.shape,
    X_test.shape,
    y_test.shape))

(40000,) (40000,)
(10000,) (10000,)


### Stochastic MAP Estimation

In [ ]:
"""
Stochastic Gradient Ascent Algorithm

1. sample data point

2. calculate predicted expectation 

3. calculate stochastic gradient

4. follow stochastic gradient (gradient step)

"""